# Digital homo- and heterodyne detection

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

import numpy as np

from qutip import *
import time
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
N=10

idc = qeye(N)
ida = qeye(2)
a  = tensor(destroy(N), ida)
adga = a.dag() * a
sm = tensor(idc, destroy(2))
sx = tensor(idc, sigmax())
sy = tensor(idc, sigmay())
sz = tensor(idc, sigmaz())


N_cav = tensor(num(N), ida)

proj0 = tensor(idc, basis(2,0).proj())
proj1 = tensor(idc, basis(2,1).proj())

initial_cavity = coherent(N,0.5)

psi0 = tensor(initial_cavity, basis(2,0))
wigner_fock_distribution(initial_cavity);

In [ ]:
def dispop(alpha):
    op = alpha * a.dag() - np.conj(alpha) * a
    return op.expm()

def interact(g):
    op = (-1j) * g * (a.dag() * sm + a * sm.dag())
    return op.expm() 

def rotateX(theta):
    op = (-1j) * (theta/2) * sx
    return op.expm()

def rotateY(theta):
    op = (-1j) * (theta/2) * sy
    return op.expm()

def rotateXY(theta,rotangl):
    op = (-1j) * (theta/2) * (np.cos(rotangl)* sx + np.sin(rotangl)* sy)
    return op.expm()

## Qubit-dyne params

In [ ]:
## Here are a few options that I tested; see how custom weights are defined below

Nmeas = 140
gint = 0.18
expspeed=0

Nmeas = 40
gint = 0.24
expspeed=1

Nmeas = 35
gint = 0.18
expspeed=2.4


In [ ]:
Nmeas = 40
gint = 0.18
expspeed=1.7

angmult=np.exp(expspeed*0.5*gint*gint*(np.linspace(0,Nmeas-1,Nmeas)))
plt.plot(angmult)

intmats=[]
for i in range(Nmeas):
    intmats.append(interact(gint*angmult[i]))

## Run simulation to extract weights

In [ ]:
# Select quadrature
rotangl = 0*np.pi/2
qbrotmat = rotateXY(np.pi/2,rotangl) # qb rotation applied prior to qb measurement, defines measured homodyne quadrature
qbfeedback = rotateX(np.pi) # this pulse will reset the qubit if found in the excited state

In [ ]:
# Find weights by solving unconditional ME

n_initial = expect(N_cav, psi0)

p1exp = []
n_cav = []
sxexp = []
rho = psi0.proj()

for i in range(Nmeas):
    
    n_cav.append((adga*rho).tr())
    intmat = interact(gint*angmult[i])
    rho = intmat * rho * intmat.dag()
    
    p1val= expect(proj1, rho)
    p1exp.append(p1val)
    
    sxval = expect(sy, rho)
    sxexp.append(sxval)
    
    rho = tensor(rho.ptrace(0),basis(2,0).proj())

## Define weights based on how much pop is extracted from cavity at each round
weights=np.array(p1exp)
weights = weights/np.sum(weights)

# This correction factor enforces that the measured expectation value of <a> agrees on the reference state
corr= np.sqrt(n_initial)/np.sum(sxexp*weights)

fig, axes = plt.subplots(1, 3, figsize=(12,3))

axes[0].plot(p1exp)
# axes[2].plot(0.5-(0.5-np.min(szexp))*np.exp(-np.linspace(0,Nmeas-1,Nmeas)*(gint*gint/2)))
lbl2 = axes[0].set_title("P1 vs iteration")

axes[1].plot(sxexp)
# axes[2].plot(0.5-(0.5-np.min(szexp))*np.exp(-np.linspace(0,Nmeas-1,Nmeas)*(gint*gint/2)))
lbl2 = axes[1].set_title("<sx> vs iteration")

axes[2].plot(n_cav)
lbl2 = axes[2].set_title("<adga> vs iteration")
plt.show()

In [ ]:
plt.plot(weights)

## Run tomography

In [ ]:
# Repeated homodyne detection: calculate single homodyne shots and collect statistics
# Weights are calculated as above

# Test Fock
psi0= tensor(basis(N,1), basis(2,0)).unit()

# Test ca
#alphain = 1 #1.41
#psi0 = (dispop(alphain) * gnd + dispop(-alphain) * gnd).unit()

Ntrajs = 1000

quadmeashom = []

xrnd = np.random.rand(Ntrajs*Nmeas)

for j in range(Ntrajs):
    
    # xrnd = np.random.rand(Nmeas)
    results = []
    psi = psi0

    for i in range(Nmeas):
        #intmat = interact(gint*angmult[i])
        intmat = intmats[i]
        psi = qbrotmat * (intmat * psi)
        # szval=(expect(sz,psi)+1)/2
        # if szval>xrnd[j*Nmeas+i]:
        prob = expect(proj0,psi)
        if prob>xrnd[j*Nmeas+i]:
            # measured g
            results.append(-1)
            psi = (proj0 * psi).unit()
        else:
            results.append(1)
            # measured e
            psi = (proj1 * psi).unit()
            # go back to g
            psi = qbfeedback * psi
    
    quadmeashom.append(np.sum(results*weights))
    # print('.')


In [ ]:
plt.hist(np.array(quadmeashom)*corr*np.sqrt(2),60,[-3,3],density=True)

k=3
x=np.linspace(-k,k,500)
plt.plot(x, np.abs((2/np.sqrt(2))*np.exp(-x**2/2)*(1/np.pi)**(1/4)*x)**2) # 1 photon